In [1]:
# Importing all the necessary libraries
import pandas as pd
import numpy as np
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Border, Side, Font
from openpyxl.utils import get_column_letter
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl.styles.alignment import Alignment
from openpyxl import Workbook
import os

In [2]:
##########Configuring all the inputs
#######Configure inputs
cwd = os.getcwd()
demand_output = "28-04-2024_Demand_Automation.xlsx"
nfc_os = os.path.join(cwd, "client_data", "NFC_OS.xlsx")
gfc_os = os.path.join(cwd, "client_data", "GFC_OS.xlsx")
kfc_os = os.path.join(cwd, "client_data", "KFC_OS.xlsx")

In [3]:
# Reading all the required sheets

# Demand Sheet
df_demand=pd.read_excel(demand_output,sheet_name='Demand_output')

# NFC Sheet
df_NFC=pd.read_excel(nfc_os,sheet_name='Available_Current_Stock_020123',skiprows=10)

# GFC Sheet
df_GFC=pd.read_excel(gfc_os,sheet_name='Available_Current_Stock_030123',skiprows=10)

# KFC Sheet
df_KFC1=pd.read_excel(kfc_os,sheet_name='KPL',skiprows=10)
df_KFC2=pd.read_excel(kfc_os,sheet_name='KPE',skiprows=10)

In [4]:
# For NFC 

# Summation of "Quantity On Hand"
df_NFCM=df_NFC.groupby(['Item Number'])['Quantity On Hand'].sum().reset_index()

# Renaming Columns
df_NFCM.rename(columns={'Item Number': 'FGSKUCode', 'Quantity On Hand': 'From NFC'}, inplace=True)

# Adding a column of FACTCode 
df_NFCM['FACTCode']='NFC'

In [5]:
# For GFC

# Summation of "Quantity On Hand"
df_GFCM=df_GFC.groupby(['Item Number'])['Quantity On Hand'].sum().reset_index()

# Renaming Columns
df_GFCM.rename(columns={'Item Number': 'FGSKUCode', 'Quantity On Hand': 'From GFC'}, inplace=True)

# Adding a column of FACTCode 
df_GFCM['FACTCode']='GFC'

In [6]:
# For KFC
df_KFC=pd.concat([df_KFC1, df_KFC2], ignore_index=True)

# Summation of "Quantity On Hand"
df_KFCM=df_KFC.groupby(['Item Number'])['Quantity On Hand'].sum().reset_index()

# Renaming Columns
df_KFCM.rename(columns={'Item Number': 'FGSKUCode', 'Quantity On Hand': 'From KFC'}, inplace=True)

# Adding a column of FACTCode 
df_KFCM['FACTCode']='KFC'

In [7]:
# From the demand sheet, filtering records where retain_flag equals 1.
df_demand=df_demand[df_demand['Ret_Flag']==1]

# Taking unique based on FGSKUCode and UM
df_FGM1OS=df_demand[['FGSKUCode', 'UM']].drop_duplicates()

# Create a new column 'WH Code' and repeat the data 3 times
df_FGM1OS['WH Code'] = df_FGM1OS.apply(lambda row: [f"NFCM", f"GFCM", f"KFCM"], axis=1)

# Explode the 'WH Code' list into separate rows
df_FGM1OS = df_FGM1OS.explode('WH Code').reset_index(drop=True)

# Adding a column of FACTCode
df_FGM1OS['FACTCode'] = df_FGM1OS['WH Code'].apply(lambda x: x.replace('M', ''))

In [8]:
# Merging df_FGM1OS (Base file) with df_NFCM based on FGSKUCode and FACTCode 
df1=pd.merge(df_FGM1OS,df_NFCM,on=['FGSKUCode','FACTCode'], how='left')

# Merging df_FGM1OS (Base file) with df_GFCM based on FGSKUCode and FACTCode
df2=pd.merge(df1,df_GFCM,on=['FGSKUCode','FACTCode'], how='left')

df_final=pd.merge(df2,df_KFCM,on=['FGSKUCode','FACTCode'], how='left')

In [9]:

# Define a function to check the conditions and assign the retain_flag
def assign_retain_flag(row):
    if np.isnan(row['From NFC']) and np.isnan(row['From GFC']) and np.isnan(row['From KFC']):
        return 0
    else:
        return 1

# Apply the function to create the 'retain_flag' column
df_final['retain_flag']=1
df_final['Missing_Flag'] = df_final.apply(assign_retain_flag, axis=1)

# Print the updated DataFrame
df_final=df_final.fillna(0)

In [10]:
# QC KPI derivation 

# Get the number of distinct values for each column
distinct_counts = df_final.nunique()
 
# Get the number of null values for each column
null_counts = df_final.isnull().sum()
 
# Get the count, mean, std, min, and percentile distribution for each column
summary_stats = df_final.describe(percentiles=[0.25, 0.5, 0.75]).transpose()
 
# Combine all the results into a single DataFrame
result_df = pd.DataFrame({
    'Count': summary_stats['count'],
    'Distinct Values': distinct_counts,
    'Null Count': null_counts,
    'Mean': summary_stats['mean'],
    'Std': summary_stats['std'],
    'Min': summary_stats['min'],
    '25%': summary_stats['25%'],
    '50%': summary_stats['50%'],
    '75%': summary_stats['75%'],
    'Max': summary_stats['max']
})

result_df=result_df.copy()
result_df['Column Name']=result_df.index
result_df=result_df[['Column Name','Count', 'Distinct Values', 'Null Count', 'Mean', 'Std', 'Min', '25%',
       '50%', '75%', 'Max']]

In [11]:
result_df

,Column Name,Count,Distinct Values,Null Count,Mean,Std,Min,25%,50%,75%,Max
FACTCode,FACTCode,NaN,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FGSKUCode,FGSKUCode,1653.0,551,0,4.003547e+09,4.301750e+07,4.001031e+09,4.001362e+09,4.001372e+09,4.001972e+09,5.011999e+09
From GFC,From GFC,1653.0,84,0,4.760194e+01,4.273271e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.978000e+03
From KFC,From KFC,1653.0,171,0,1.029868e+02,8.460194e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.200000e+04
From NFC,From NFC,1653.0,236,0,3.347456e+02,2.121523e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.333955e+04
Missing_Flag,Missing_Flag,1653.0,2,0,3.157895e-01,4.649702e-01,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00
UM,UM,NaN,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WH Code,WH Code,NaN,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
retain_flag,retain_flag,1653.0,1,0,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [12]:
# Output Formatting Function
def apply_formatting(worksheet):
    # Remove gridlines
    worksheet.sheet_view.showGridLines = False

    # Add one row and column at the top
    worksheet.insert_rows(1)
    worksheet.insert_cols(1)
    worksheet.cell(row=1, column=1)

    # Add table borders
    border = Border(left=Side(style='thin'), 
                    right=Side(style='thin'), 
                    top=Side(style='thin'), 
                    bottom=Side(style='thin'))
    for row in worksheet.iter_rows(min_row=2, min_col=2, max_col=worksheet.max_column):
        for cell in row:
            cell.border = border

    # Set light blue color for headers
    for cell in worksheet.iter_cols(min_row=2, min_col=2):
        cell[0].fill = PatternFill(start_color="ADD8E6", end_color="ADD8E6", fill_type="solid")  # Light blue
    
    # Remove borders for the extra added row and column at the top
    for cell in worksheet['A1:C1'][0]:
        cell.border = None

    # Autofit columns
    for col in worksheet.columns:
        max_length = 0
        column = col[0].column_letter  # Get the column name
        for cell in col:
            try:  # Necessary to avoid error on empty cells
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        adjusted_width = (max_length + 2) * 1.2
        worksheet.column_dimensions[column].width = adjusted_width

    # Apply auto filters starting from the 2nd row in the B column
    last_row = worksheet.max_row
    last_column = worksheet.max_column
    range_str = f"B2:{get_column_letter(last_column)}{last_row}"
    worksheet.auto_filter.ref = range_str

def save_dataframes_to_excel(dataframes_list, file_paths_list):
    for dataframes, file_path in zip(dataframes_list, file_paths_list):
        with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
            for sheet_name, dataframe in dataframes.items():
                dataframe.to_excel(writer, sheet_name=sheet_name, index=False)

            workbook = writer.book
            for sheet_name in dataframes.keys():
                worksheet = workbook[sheet_name]
                apply_formatting(worksheet)

            workbook.save(file_path)


In [13]:
dataframes_list = [
    {'FG-M1OS': df_final, 'QC KPI': result_df}
]

file_paths_list = [
    '29-04-24_FG M10S Output.xlsx',
]

# Call the function to save DataFrames to Excel with formatting
save_dataframes_to_excel(dataframes_list, file_paths_list)

print("All DataFrames are saved to {}".format(file_paths_list))

All DataFrames are saved to ['29-04-24_FG M10S Output.xlsx']
